# Part 2: Creating a Basic Agent

![Alt text](img/augLLMs.png)

Let's walk through the fundamentals of building a basic agent.

At a high level, agents aren't magical — they are just LLMs augmented with the ability to call external tools when needed.  
In fact, a very simple agent is often just **an LLM in a while loop**, deciding when to call tools based on what the user asks.

The key concepts we'll cover:
- **Function Definition**: What tools the LLM can access.
- **Function Call**: A special output from the model indicating that a tool should be used.
- **Function Response**: The result we get from executing that tool.

To tie all of this together, we'll build a minimal **LLM Framework** that wraps around the model's outputs.

We'll also relate this to a broader view:  
In more complex systems, agents often combine **retrieval**, **tools**, and **memory** — but at the core, the pattern remains the same:  
The LLM outputs a call, and the framework decides what to do next.

![functioncalling](img/function-calling-overview.png)

## Our First LLM Tool

Before we build our agent, we need at least one tool that the LLM can call.

This is a simple (and arbitrary) tool — but the same pattern applies to any function you might want the model to use.

We'll start by building a very basic weather lookup function, because it's intuitive and lets us focus on the function call mechanics rather than the tool itself.

In [4]:
import logging
import re
import json

def get_weather(city):
    logging.info("Called weather function %s", city)
    if city.lower() in {"austin", "sydney"}:
        return "sunny"
    else:
        return "cloudy"

In [6]:
get_weather("Austin")

'sunny'

## Sending a Basic Prompt to the Model

Now that we have a tool ready, let's set up a simple function to interact with the model.

We'll define a basic `model_call(prompt)` function that sends user input to the LLM and receives a response.  
This will be the foundation we build on as we add more complex behaviors like tool calling.

In [14]:
from ollama import chat
from ollama import ChatResponse

# 12b is better so try and use it first
model = 'gemma3:4b'


# Note, the argument model_prompt is specific here
def model_call(model_prompt):
    
    response: ChatResponse = chat(model=model, messages=[
      {
        'role': 'user',
        'content': model_prompt,
      },
    ])
    return response['message']['content']

user_prompt = "Say hello to the class"

# Note, the argument user_prompt is specific here
model_call(user_prompt)

'Hello everyone! 😊 \n\nIt’s great to be here with you all today. \n\nHow’s everyone doing?'

## Guiding the Model with a System Prompt

System prompts let us control how the model behaves before it sees the user's input.  
We'll define a simple function that prepends a system instruction to the user's message, allowing us to guide tone, style, or available tools.


In [15]:
def augmented_model_call(system_prompt, user_prompt, print_prompt = False):
    combined_prompt = f"{system_prompt}\n{user_prompt}"

    if print_prompt:
        print(combined_prompt)
    
    return model_call(combined_prompt)

In [16]:
# Example system prompt: talk like a pirate
# This is injected by the LLM application behind the scenes

system_prompt = '''Talk like a pirate to everyone. \n '''
user_prompt = "Say hello to the class"

augmented_model_call(system_prompt, user_prompt, print_prompt=True)

Talk like a pirate to everyone. 
 
Say hello to the class


"Arrr, me hearties! \n\n**Hello to ye all!** Shiver me timbers, it's good to be seein' yer faces! Let's have a grand time, aye? \n\nNow, let's get to it, savvy? 🏴\u200d☠️ \n\n---\n\nHow's it goin' with ye all? Do ye have any questions for a salty old pirate like meself?"

## Teaching the Model to Suggest Tool Calls

Now that we've seen how system prompts can guide behavior, we'll write a system prompt that teaches the model how to suggest when a tool should be used.  
We'll structure the prompt so that the model outputs a JSON instruction when it decides a function call is needed.

In [19]:
system_prompt = '''
You have the following functions available
 def get_weather(city: str)
   """Given a city returns the weather for that city""

 If you call this function return the json [{"city": city}] and nothing else
 otherwise respond normally
'''

In [20]:
user_prompt = "What's the weather in Sydney?"
augmented_model_call(system_prompt, user_prompt)

'```json\n{"city": "Sydney"}\n```\n'

In [21]:
user_prompt = "How's the Austin forecast looking today?"
augmented_model_call(system_prompt, user_prompt)

'```json\n{"city": "Austin"}\n```'

In [22]:
user_prompt = "How's the London weather looking today?"
augmented_model_call(system_prompt, user_prompt)

'```json\n{"city": "London"}\n```\n'

In [24]:
user_prompt = "Say hi to Hugo"
augmented_model_call(system_prompt, user_prompt)

'Hi Hugo!\n'

## Parsing the Response

Now that the model knows how to suggest tool calls, we need to build a small framework around it to actually process results.

The basic logic we want:

1. If there is **no function call**, simply return the model's response to the user.
2. If there **is a function call**:
   - a. Call the appropriate tool to get new information.
   - b. Either return the tool result directly, or reinject it back into the model to generate a better final response.

We'll start by writing a simple function to detect when the model outputs a tool call in JSON format.

In [36]:
pattern = f"```json\n(.*?)\n``"

def parse_response(model_response):
    if tool_call := re.search(pattern, model_response):
        #import pdb; pdb.set_trace()
        return json.loads(tool_call.groups(0)[0])
    return None

model_response = '```json\n[{"city": "Austin"}]\n```\n'
# parse_response(model_response)

In [28]:
model_response = 'hi! Hugo'
print(parse_response(model_response))

None


## Putting It All Together: Building Our First Agent

Now that we've built all the individual pieces, let's combine them to create our first real agent.  
This agent will:
- Send a prompt to the model.
- Detect if a tool call is needed.
- Call the tool and get the result.
- Optionally feed the tool result back into the model for a better final answer.

In [39]:
def chat_interaction(user_prompt):
    system_prompt = '''
    You have the following functions available:

    def get_weather(city: str)
    """Given a city returns the weather for that city"""

    If you call this function return the json [{"city": city}] and nothing else
    otherwise respond normally
    '''

    # Get a model response. Right now we don't know if it's a function call or chat response
    model_response = augmented_model_call(system_prompt, user_prompt)

    # Regex to see if we have the json which indicates a function call
    function_call_json = parse_response(model_response)

    # If it's not a function call, return the response
    if not function_call_json:
        return model_response
    
    # Since we detect a function call
    weather = get_weather(function_call_json["city"])

    # We have a choice here
    # We could return the weather directly to the user
    # But for a nicer experience let's reinject it into the LLM for a better final response
    function_response_prompt = f"The weather in {function_call_json['city']} is {weather}, tell me the weather as if you were a pirate"

    # We already checked for weather so we don't need to go again
    model_response = model_call(function_response_prompt)
    return model_response

chat_interaction("Can you say hi to Hugo?")

'Hi Hugo!\n'

In [40]:
chat_interaction("What's the weather today in London?")

"Shiver me timbers! Blast it all, the skies o' London be a right grey mess, aye! Thick as pea soup, they are, a proper blanket o' clouds hangin' low. Not a lick o' sunshine to be seen, just a damp, dreary gloom. \n\nThe wind's a bit cheeky too, a salty breeze blowin' in, carryin' a bit o' drizzle.  It's the kind o' weather that'd make a seasoned sailor long for a rum and a dry deck! \n\nBest keep yer oilskins handy, matey!  It's a miserable day for sailin', that's for sure! \n\nArrr!"

In [37]:
chat_interaction("How are things looking in Austin?")

"Okay, let's see... it's a beautiful, sunny day in Austin! It’s absolutely lovely out there – perfect for enjoying the sunshine. 😊 \n\nWould you like me to tell you anything more about the weather, like the temperature or what to wear?"

## 🎯 Recap: What We Learned

In this section, we built our first basic agent that can recognize when a tool call is needed and respond accordingly.

Here are the key ideas to remember:
- **Function calls aren't magic**: We rely on the model's "reasoning" to decide when to call a function, based on the system prompt and user input.
- **The model doesn't actually call functions itself**: It simply outputs a structured signal (like JSON) suggesting what should happen.  
- **The framework — your code — decides what to do**: It parses the model’s output, calls tools when needed, and can reinject the results for a better final answer.

This pattern — LLM suggests, framework acts — is the foundation for building more complex agents later.

## Your Turn: Connect to a Real API

Now that you've seen how we can define simple tools manually,  
let's try using a real external API to fetch live data.

Below is a starting point — your task is to plug this into an agent loop, just like we did earlier with our fake `get_weather` function.

In [19]:
import requests

def get_weather(latitude, longitude):
    response = requests.get(f"https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}&current=temperature_2m,wind_speed_10m&hourly=temperature_2m,relative_humidity_2m,wind_speed_10m")
    data = response.json()
    return data['current']['temperature_2m']

Let's test it by fetching the current temperature for London!

In [20]:
LONDON_LATITUDE = 51.5074 # Approximate latitude for London
LONDON_LONGITUDE = -0.1278 # Approximate longitude for London (West is negative)
get_weather(LONDON_LATITUDE, LONDON_LONGITUDE)

9.8

## Tool Call versus Agents

![functioncalling](img/AlwaysHasBeen.jpg)
Source: https://huggingface.co/blog/tiny-agents


Agents have a lot of definitions.  
The simplest is that they're a model that can call tools inside a while loop.

The model suggests an action, but it's our code that decides what actually happens.